Airline AI Assistant - this is an extension of a previous project "Airline Agent". 
- In this notebook I will be creating another Agent to Translate all the responses into a different language.
- Adding more tools to handle booking a flight

In [356]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

Load in the API keys

In [357]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


System Prompts

In [379]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "If a flight is not available on a certain date, say that it is not available."
# system_message += "If a flight is available on a certain date, and the customer requests to book it, say here is your ticket, have a nice flight."

# This last system message is a good way to ensure the agent does not hallucinate

Forcing dark mode in Gradio

In [380]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

#### Tools
- Tools are an incredibly powerful feature provided by the frontier LLMs.
- With tools, you can write a function, and have the LLM call that function as part of its response.
- Sounds almost spooky.. we're giving it the power to run code on our machine? Well, kinda.

In [381]:
ticket_prices = {
    "london": "$799", 
    "paris": "$899", 
    "tokyo": "$1400", 
    "berlin": "$499"
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [382]:
ticket_availability = {
    "london": ["11th", "12th"],
    "paris": ["13th", "14th"],
    "tokyo": ["15th", "16th"],
    "berlin": ["13th", "14th"]
}


def get_ticket_availability(destination_city):
    print(f"Tool get_ticket_dates called for {destination_city}")
    city = destination_city.lower()
    return ticket_availability.get(city, "Unknown")

In [383]:
# import random

# def book_flight(destination_city):
#     print(f"Tool to book a flight for {destination_city}")
#     city = destination_city.lower()
#     availability = ticket_availability.get(city, "Unknown")
#     price =ticket_prices.get(city, "Unknown")
    
#     confirmation_number = f"FLIGHT-{random.randint(100000, 999999)}"
    
#     return {
#         "status": "success",
#         "confirmation_number": confirmation_number,
#         "destination_city": destination_city,
#         "ticket_availability": availability,
#         "ticket_price": price
#     }

In [424]:
import random

def book_flight(destination_city, travel_date):
    print(f"Tool to book a flight for {destination_city} on {travel_date}")
    city = destination_city.lower()
    availability = get_ticket_availability(city)
    price = get_ticket_price(city)
    if travel_date not in availability:
        return {
            "status": "error",
            "message": f"No flights available for {destination_city} on {travel_date}."
        }

    
    confirmation_number = f"FLIGHT-{random.randint(100000, 999999)}"

    return {
        "status": "success",
        "confirmation_number": confirmation_number,
        "destination_city": destination_city,
        "ticket_availability": availability,
        "ticket_price": price
    }


In [425]:
book_flight("Berlin", "13th")

Tool to book a flight for Berlin on 13th
Tool get_ticket_dates called for berlin
Tool get_ticket_price called for berlin


{'status': 'success',
 'confirmation_number': 'FLIGHT-230820',
 'destination_city': 'Berlin',
 'ticket_availability': ['13th', '14th'],
 'ticket_price': '$499'}

In [426]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [427]:
get_ticket_availability("Berlin")

Tool get_ticket_dates called for Berlin


['13th', '14th']

In [428]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [429]:
availability_function = {
    "name": "get_ticket_availability",
    "description": "Call this whenever you need to know the date of a flight to a particular city, for example when a customer asks 'Are there any available flights to this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [430]:
book_flight_function = {
    "name": "book_ticket",
    "description": "Book a ticket to a destination city on a specific date.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the customer wants to travel to",
            },
            "travel_date": {
                "type": "string",
                "description": "The date the customer wants to travel on (e.g., '13th')",
            },
            "ticket_price": {
                "type": "string",
                "description": "The price of the ticket being booked.",
            }
        },
        "required": ["destination_city", "travel_date", 'ticket_price'],
        "additionalProperties": False
    }
}


In [431]:
tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": availability_function},
         {"type": "function", "function": book_flight_function}]

#### Getting OpenAI to use our Tool
- There's some fiddly stuff to allow OpenAI "to call our tool"
- What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.
- Here's how the new chat function looks:

In [440]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response, city
    
    elif tool_call.function.name == "get_ticket_availability":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        availability = get_ticket_availability(city)  
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "availability": availability}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response, city
    
    elif tool_call.function.name == "book_flight":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        travel_date = arguments.get('travel_date')  # Extract the travel date
        
        availability = ticket_availability.get(city, [])
        
        # Check if the date is valid
        if travel_date not in availability:
            response = {
                "role": "tool",
                "content": json.dumps({
                    "status": "error",
                    "message": f"No flights available for {city.title()} on {travel_date}."
                }),
                "tool_call_id": message.tool_calls[0].id
            }
            return response, city
        
        price = get_ticket_price(city)
        confirmation_number = f"FLIGHT-{random.randint(100000, 999999)}"
        
        response = {
            "role": "tool",
            "content": json.dumps({
                "status": "success",
                "destination_city": city,
                "travel_date": travel_date,
                "ticket_price": price,
                "confirmation_number": confirmation_number
            }),
            "tool_call_id": message.tool_calls[0].id
        }
        return response, city



#### Chat Function
- we have added in two new message lines to allow our agent to use this tool
    - adding in print statments could make this more clear

- messages = response.choices[0].message 
    - This is nothing more than what we get back from our Assistant asking to run a tool
- response, city = handle_tool_call(message)
    - This is the result of calling the tool function

In [441]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message 
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [443]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

# Interesting, I get an error when I don't specify the city name when asking for a price

# Interesting, I get an error when asking to book a flight even after being told the availability
# I also receive an error when asking to the book a flight to Berlin on the 13th.

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_dates called for Berlin
Tool get_ticket_price called for Berlin


Traceback (most recent call last):
  File "c:\Users\Erica\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Erica\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Erica\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Erica\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1587, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Erica\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py", line 850, in async_wrapper
    response = await f(*args, **kwargs)
              

Talker Function

In [435]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

In [436]:
# def chat(history):
#     messages = [{"role": "system", "content": system_message}] + history
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
#     image = None
    
#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         image = artist(city)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
        
#     reply = response.choices[0].message.content
#     history += [{"role":"assistant", "content":reply}]

#     # Comment out or delete the next line if you'd rather skip Audio for now..
#     talker(reply)
    
#     return history, image

In [437]:
# gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

In [438]:
# # More involved Gradio code as we're not using the preset Chat interface!
# # Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

# with gr.Blocks(js=force_dark_mode) as ui:
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500, type="messages")
#         image_output = gr.Image(height=500)
#     with gr.Row():
#         entry = gr.Textbox(label="Chat with our AI Assistant:")
#     with gr.Row():
#         clear = gr.Button("Clear")

#     def do_entry(message, history):
#         history += [{"role":"user", "content":message}]
#         return "", history

#     entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
#         chat, inputs=chatbot, outputs=[chatbot, image_output]
#     )
#     clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# ui.launch(inbrowser=True)

Tool get_ticket_dates called for Berlin
Tool get_ticket_price called for Berlin
